Harsha Pukkalla and Kaitlynn Whitney

In [ ]:
!git clone https://github.com/kawhitney/CSCE4201_Project1.git

fatal: destination path 'CSCE4201_Project1' already exists and is not an empty directory.


In [ ]:
from CSCE4201_Project1.vehicle import Vehicle, Customer

from IPython.display import clear_output
import networkx as nx
from networkx.classes.function import neighbors
import pandas as pd
import random
# import sys
import time

In [ ]:
# create fleet
def create_fleet(fleet_size, locationInfo):
  fleet = []
  positions = randomGenerator(0, 199, fleet_size+1)
  for i in range(fleet_size):
    fleet.append(Vehicle(i, positions[i]))
    if positions[i] in locationInfo.keys():
      locationInfo[positions[i]].append("V"+str(i))
    else:
      locationInfo[positions[i]] = ["V"+str(i)]
  return fleet

# move vehicles in fleet
def move_vehicle(vehicle, locationInfo):
  if len(vehicle.path) > 0:
    locationInfo[vehicle.currentPosition].remove("V"+str(vehicle.vehicleID))

    if len(locationInfo[vehicle.currentPosition]) == 0: 
       del(locationInfo[vehicle.currentPosition])

    # print(locationInfo)
    vehicle.move_position(vehicle.path.pop(0))
    if vehicle.currentPosition in locationInfo.keys():
      locationInfo[vehicle.currentPosition].append("V"+str(vehicle.vehicleID))
    else:
      locationInfo[vehicle.currentPosition] = ["V"+str(vehicle.vehicleID)]
    
    return True
  else:
    return False

# update vehicle destination
def update_vehicle_destination(vehicle):
  vehicle.destination = vehicle.path[-1]

# add on to vehicle path
def add_to_vehicle_path(vehicle, pickup, graph, resultfile):
  if len(vehicle.path) > 0:
    original_destination = vehicle.destination
    vehicle.destination = pickup
    # path to pick up customer
    path1 = nx.dijkstra_path(graph, vehicle.currentPosition, vehicle.destination)
    # resume original drop off destination
    path2 = nx.dijkstra_path(graph, vehicle.destination, original_destination)
    #combine paths (vehicle will pick up new customer then go resume dropoff of other customer)
    path = path1 + path2[1:]
  else:
    vehicle.destination = pickup
    path = nx.dijkstra_path(graph, vehicle.currentPosition, pickup)
  #print(f"{vehicle.vehicleID}->{path}")
  resultfile.write(f"{vehicle.vehicleID}->{path}\n\n")
  vehicle.set_path(path[1:])

# create customer
def create_customer(customerID, locationInfo, graph):
  pickup = random.randint(0, 199)
  dropoff = random.randint(0, 199)
  # travel length greater than or equal to 10
  while nx.dijkstra_path_length(graph, pickup, dropoff) < 9:
    dropoff = random.randint(0, 199)  
  if pickup in locationInfo.keys():
    locationInfo[pickup].append("C"+str(customerID))
  else:
    locationInfo[pickup] = ["C"+str(customerID)]

  return Customer(customerID, pickup, dropoff)

# probability of customer generating
def customer_probability(event, outcomes):
  probability=event/outcomes
  return random.random() < probability

# random position generator
def randomGenerator(start, end, size):
  randomnumbers = []
  if size==1:
    return random.randint(start, end)
  else:
    for i in range(size):
      currentnum = random.randint(start, end)
      randomnumbers.append(currentnum)
    return randomnumbers

# create dataframe
def create_dataframe(fleet):
  columns = ['Miles Traveled', 'Number of Customers', 'Current Location' , 'Desitnation'] # 'Path']
  rows = []
  data = []
  for vehicle in fleet:
    rows.append(vehicle.vehicleID)
    data.append([vehicle.miles, len(vehicle.customers), vehicle.currentPosition, vehicle.destination]) 
  return pd.DataFrame(data=data, index=rows, columns=columns)


# **Q3:**

In [ ]:
def requestPickup(customer, locationInfo, map, vehicles):
  availableVehicle = None
  #List to store all the nodes that need to be checked for any available vehicles
  checkloc = []
  #List to store all the visited nodes
  visited = []
  #Variable to store the presentlocation of the vehicle
  presentlocation = None
  #Variable to store the estimated pickup time 
  eta = 0

  #Scenario where the vehicle is immediately present at the customer's pickup location
  for i in locationInfo[customer.pickup]:
    if "V" in i:
      v = vehicles[int(i[1:])]
      # Checking if the vehicle has vacant seats
      if len(v.customers)<= 4 and v.is_busy == False: # will be max of 5 if length = 4 and is_busy is True
        availableVehicle = v.vehicleID
        v.add_customer(customer)
        presentlocation = customer.pickup
        eta = 0
        break
        
  #Scenario where the vehicle is not available at the customer's pickup location
  if availableVehicle == None:
    #Adding the pickup node to checkloc as it was checked already      
    checkloc.append(customer.pickup)
    #Adding the pickup node to visited as it was visited already
    visited.append(customer.pickup)

    #This loop runs until either a vehicle is found  or there are no more nodes left in the map to check
    while availableVehicle == None and len(checkloc) != 0:
      #Initially, this stores all the neighbors of the pickup node into checkloc
      #At each iteration it stores the neighbors of the next neighbor at the end of checkloc excluding the nodes already visited
      for i in map.neighbors(checkloc[0]):
        if i not in visited:
          checkloc.append(i)

      #Deleting the already visited node      
      del checkloc[0]
      if len(checkloc) == 0:
        break
      #checkloc[0] now contains the immediate neighbor
      if checkloc[0] in locationInfo.keys():
        for j in locationInfo[checkloc[0]]:
          if "V" in j:
            v = vehicles[int(j[1:])]
            # Checking if the vehicle has vacant seats and is not currently picking up customer
            if len(v.customers)<=5 and v.is_busy == False: 
              # change the busy status of the vehicle
              v.change_busy() 
              availableVehicle = v.vehicleID
              # set customer in vehicle queue
              v.customer_in_queue = customer
              presentlocation = i
              distance = nx.dijkstra_path_length(map, customer.pickup, i)
              eta = distance*6
              break
      #Adding the current node to visited        
      visited.append(checkloc[0]) 
  return availableVehicle, eta


In [ ]:
# find drop off path
def drop_off_path(vehicle, graph, resultfile):
  dropoff = []
  if len(vehicle.customers) > 0:
    for customer in vehicle.customers:
      path = nx.dijkstra_path(graph, vehicle.currentPosition, customer.dropoff)
      if len(path) < len(dropoff) or len(dropoff) == 0:
        dropoff = path[:]
  if len(dropoff) != 0:   
    vehicle.set_path(dropoff)
    resultfile.write(f"Path for vehicle {vehicle.vehicleID}: {vehicle.path}\n\n")
    vehicle.destination=vehicle.path[-1]

# customer drop off
def customer_drop_off(vehicle):
  for customer in vehicle.customers:
    if customer.dropoff == vehicle.currentPosition:
      vehicle.remove_customer(customer)

# **Q4:**

In [ ]:
def rideshare(nodes, connectivity, fleet, filename):
  
  resultfile = open(filename, 'w')

  locationInfo = {}
  map = nx.connected_watts_strogatz_graph(nodes, connectivity, 0, tries=10, seed=1000) 
  vehicles = create_fleet(fleet, locationInfo)

  hours = 8
  while(hours > 0):
    resultfile.write("\n"+"="*20 + f"Hour: {9-hours}" + "="*20 + "\n\n")
    hours -= 1
    hasCustomer = False 
    # 100 to 150 customers in the hour
    numCustomers = random.randrange(100, 150)
    # execute the commands for the hour
    for i in range(1200): # 1 hour = 60 min = 1200 sec
      # add loop break for when there are no more customers left
      if customer_probability(numCustomers, 1200-i) is True and numCustomers > 0: 
        if hasCustomer == False:
          hasCustomer = True
        # generate customer
        customer = create_customer(numCustomers, locationInfo, map)
        resultfile.write(f"Customer {customer.customerID} requests pickup at {customer.pickup}.")
        numCustomers -= 1
        # find pick up vehicle
        assignedvehicle, eta = requestPickup(customer, locationInfo, map, vehicles)
        resultfile.write(f"Vehicle {assignedvehicle} has been assigned to customer {customer.customerID} with an eta of {eta} seconds.\n")
        # append the path from assigned vehicle destination to customer pickup
        add_to_vehicle_path(vehicles[assignedvehicle], customer.pickup, map, resultfile)
      # 30mph = 0.00833 mps => .05/.00833 = 6 sec
      if i%6 == 0 and hasCustomer == True: # vehicles will move every 6 seconds
        for vehicle in vehicles:
          if move_vehicle(vehicle, locationInfo):
            vehicle.miles += 0.05
          if vehicle.currentPosition == vehicle.destination and (len(vehicle.customers) > 0 or vehicle.customer_in_queue != None): 
            if vehicle.is_busy == True:
              resultfile.write(f"Customer {vehicle.customer_in_queue.customerID} has been picked up by {vehicle.vehicleID}\n")
              # pick up customer
              vehicle.add_customer(vehicle.customer_in_queue)
              vehicle.customer_in_queue = None
              vehicle.change_busy()
            else:
              # drop off customer
              customer_drop_off(vehicle)
            # recalculate vehicles travel path if none exists
            if len(vehicle.path) == 0:
              drop_off_path(vehicle, map, resultfile)
            else:
              update_vehicle_destination(vehicle)
        # update the dataframe variables
        df = create_dataframe(vehicles)

    #The contents of the dataframe are not being stored in the text file. Will check on how this can be done
    resultfile.write(df.to_string())

  totalMiles = 0
  for vehicle in vehicles:
    totalMiles += vehicle.miles
  print(f"Total Miles Traveled: {totalMiles} miles")
  resultfile.write(f"Total Miles Trave2led: {totalMiles} miles\n")
  print(f"Average Miles Traveled: {totalMiles/fleet}")
  resultfile.write(f"Average Miles Traveled: {totalMiles/fleet}")



In [ ]:
nodes = int(input("Enter the number of nodes in the graph: "))
connectivity = int(input("Enter the average connectivity between the nodes: "))
fleet = int(input("Enter the fleet size: "))

rideshare(nodes, connectivity, fleet, 'rideshareresultQ4.txt')

Enter the number of nodes in the graph: 200
Enter the average connectivity between the nodes: 2
Enter the fleet size: 30
Total Miles Traveled: 1947.3999999999542 miles
Average Miles Traveled: 64.9133333333318


# **Q5:**

In [ ]:
nodes = int(input("Enter the number of nodes in the graph: "))
connectivity = int(input("Enter the average connectivity between the nodes: "))
fleet = int(input("Enter the fleet size: "))

rideshare(nodes, connectivity, fleet, 'rideshareresultQ5.txt')

Enter the number of nodes in the graph: 200
Enter the average connectivity between the nodes: 2
Enter the fleet size: 60
Total Miles Traveled: 2409.699999999988 miles
Average Miles Traveled: 40.16166666666647


# **Q6:**

In [ ]:
nodes = int(input("Enter the number of nodes in the graph: "))
connectivity = int(input("Enter the average connectivity between the nodes: "))
fleet = int(input("Enter the fleet size: "))

rideshare(nodes, connectivity, fleet, 'rideshareresultQ6.txt')

Enter the number of nodes in the graph: 200
Enter the average connectivity between the nodes: 4
Enter the fleet size: 30
Total Miles Traveled: 1411.8499999999842 miles
Average Miles Traveled: 47.06166666666614
